In [1]:
import pandas as pd
import numpy as np
import warnings
import zipfile
import regex as re
warnings.filterwarnings('ignore')

In [2]:
zf = zipfile.ZipFile('/Users/borja/IronHack/Proyecto-Sharks_Attacks/data/Archive.zip')


archivo = zf.open('attacks.csv')


data_original = pd.read_csv(archivo, encoding='latin1')
pd.set_option('display.max_columns', None)

In [3]:
data_original.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [4]:
data = data_original.copy()

In [5]:
data.shape

(25723, 24)

In [6]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [7]:
duplicados = data[data.duplicated()] #borramos las filas publicadas
len(duplicados)

19411

In [8]:
data = data.drop_duplicates() #tenemos un nuevo DF con 19.411 filas borradas
data.shape

(6312, 24)

In [9]:
data.isna().sum()

Case Number                  2
Date                        10
Year                        12
Type                        14
Country                     60
Area                       465
Location                   550
Activity                   554
Name                       220
Sex                        575
Age                       2841
Injury                      38
Fatal (Y/N)                549
Time                      3364
Species                   2848
Investigator or Source      27
pdf                         10
href formula                11
href                        10
Case Number.1               10
Case Number.2               10
original order               3
Unnamed: 22               6311
Unnamed: 23               6310
dtype: int64

In [10]:
data.reset_index(drop=True, inplace=True)
filas_de_nulos = data.isna().mean(axis=1) > 0.6

In [11]:
data[filas_de_nulos].shape #solo hay 10 filas con nulos. no borramos por no ser significativo

(10, 24)

In [12]:
data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [13]:
data.columns = [c.lower().strip().replace(' ', '_') for c in data.columns]

In [14]:
data.columns

Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal_(y/n)', 'time',
       'species', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number.1', 'case_number.2', 'original_order', 'unnamed:_22',
       'unnamed:_23'],
      dtype='object')

# Case Number

In [15]:
data.case_number.value_counts()

0               8
1920.00.00.b    2
1966.12.26      2
2014.08.02      2
1990.05.10      2
               ..
1999.09.05      1
1999.09.10      1
1999.09.16      1
1999.09.18      1
xx              1
Name: case_number, Length: 6287, dtype: int64

In [16]:
data.case_number = data.case_number.str.strip()

In [17]:
def filtrar_fechas(texto):
    # Definir el patrón de fecha: año.mes.día
    texto = str(texto)
    patron_fecha = r'\b\d{4}\.\d{2}\.\d{2}\b'
    
    
    
    # Buscar todas las fechas que coincidan con el patrón en el texto
    fechas = re.findall(patron_fecha, texto)

    # Unir todas las fechas encontradas en una sola cadena de texto
    fechas_string = ' '.join(fechas) 

    
    return fechas_string

In [18]:
data.case_number = data.case_number.apply(filtrar_fechas) #aplicamos la función anterior y corregimos fechas

In [19]:
data.case_number.value_counts()

              142
1950.00.00     13
1942.00.00     11
1957.00.00     11
1960.00.00     10
             ... 
1995.06.23      1
1995.06.24      1
1995.06.30      1
1995.07.00      1
0005.00.00      1
Name: case_number, Length: 5200, dtype: int64

In [20]:
#todo lo que venga con fechas anteriores a 1970 lo eliminamos. 
#No vamos a trabajar con fecha anteriores a los último 50 años
def filtrar_fechas2(texto):
    pattern = r'(\d{4})\.(\d{2})\.(\d{2})'
    match = re.search(pattern, texto)
    if match:
        year, month, day = match.groups()
        if int(year) < 1970:
            return np.nan
        if month == '00':
            month = '01'
        if day == '00':
            day = '01'
        return f'{year}.{month}.{day}'
    else:
        return np.nan


In [21]:
data.case_number = data.case_number.apply(filtrar_fechas2)

In [22]:
data.case_number.value_counts()

1970.01.01    9
1972.01.01    5
2003.10.05    5
1995.07.28    5
2001.04.12    5
             ..
2007.12.09    1
2007.12.07    1
2007.11.18    1
2007.11.08    1
2003.10.14    1
Name: case_number, Length: 2821, dtype: int64

In [23]:
data.case_number = pd.to_datetime(data.case_number,format='%Y-%m-%d')

In [24]:
data.case_number.head()

0   2018-06-25
1   2018-06-18
2   2018-06-09
3   2018-06-08
4   2018-06-04
Name: case_number, dtype: datetime64[ns]

# Date

In [25]:
data.date.value_counts()

1957                          11
1942                           9
1956                           8
1958                           7
1950                           7
                              ..
01-Dec-1994                    1
Reported      10-Dec-1994      1
11-Dec-1994                    1
13-Dec-1994                    1
1845-1853                      1
Name: date, Length: 5433, dtype: int64

In [26]:
data.date = data.date.str.strip()
data.date = data.date.str.lstrip('Reported ')

In [27]:
data.date = pd.to_datetime(data.date,errors='coerce',format='%Y-%m-%d')

In [28]:
data.date.isnull().value_counts()

True     6020
False     292
Name: date, dtype: int64

In [29]:
data.date.value_counts()

1957-01-01    11
1942-01-01     9
1956-01-01     8
1950-01-01     7
1958-01-01     7
              ..
1918-01-01     1
1919-01-01     1
1925-01-01     1
1932-01-01     1
1723-01-01     1
Name: date, Length: 133, dtype: int64

In [30]:
data[['case_number','date']].head(15)

,case_number,date
0,2018-06-25,NaT
1,2018-06-18,NaT
2,2018-06-09,NaT
3,2018-06-08,NaT
4,2018-06-04,NaT
5,2018-06-03,NaT
6,2018-06-03,NaT
7,2018-05-27,NaT
8,2018-05-26,NaT
9,2018-05-26,NaT


In [31]:
indice_nulos = data[data.case_number.isna()].index
data.loc[indice_nulos,'case_number'] = data.loc[indice_nulos,'date']

In [32]:
data.case_number.isnull().value_counts()

False    3650
True     2662
Name: case_number, dtype: int64

In [33]:
data.date = data.case_number
data['case_number.1'] = data.case_number
data['case_number.2'] = data.case_number

# Type

In [34]:
data.type.value_counts()

Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: type, dtype: int64

In [35]:
data.type = data.type.str.replace('Boatomg', 'Boat')

In [36]:
data.type = data.type.str.replace('Boating', 'Boat')

In [37]:
data.type = data.type.str.replace('Questionable', 'Other')

In [38]:
data.type = data.type.str.replace('Invalid', 'Other')

In [90]:
data.type = data.type.str.upper()

In [39]:
data.type.value_counts()

Unprovoked      4595
Provoked         574
Other            549
Boat             341
Sea Disaster     239
Name: type, dtype: int64

# Country

In [40]:
len(data.country.unique())

213

In [41]:
data.country = data.country.str.upper()
data.country = data.country.str.replace(r'[(\)\-\?]+', ' ')
data.country = data.country.str.replace('CEYLON  ','')
data.country = data.country.replace(['NORTH SEA', 'RED SEA / INDIAN OCEAN', 'BRITISH WEST INDIES', 'ASIA', 'COAST OF AFRICA', 'WEST INDIES', 'CENTRAL PACIFIC', 'RED SEA', 'MID ATLANTIC OCEAN', 'SOUTH PACIFIC OCEAN', 'BETWEEN PORTUGAL & INDIA', 'MEDITERRANEAN SEA'], np.nan)
data.country = data.country.str.strip()
len(data.country.unique())

188

In [42]:
data.country.value_counts()

USA                           2229
AUSTRALIA                     1338
SOUTH AFRICA                   579
PAPUA NEW GUINEA               134
NEW ZEALAND                    128
                              ... 
MAYOTTE                          1
GABON                            1
ANDAMAN / NICOBAR ISLANDAS       1
MALDIVE ISLANDS                  1
ASIA                             1
Name: country, Length: 187, dtype: int64

# Area

In [43]:
len(data.area.unique())

826

In [44]:
data.area = data.area.str.upper()
data.area = data.area.str.strip()

In [45]:
data.area.value_counts()

FLORIDA                     1038
NEW SOUTH WALES              486
QUEENSLAND                   312
HAWAII                       298
CALIFORNIA                   290
                            ... 
REGGIO CALABRIA PROVINCE       1
GENOA PROVINCE                 1
GANGES-BRAHMAPUTRA DELTA       1
AULONG ISLAND                  1
MOALA ISLAND                   1
Name: area, Length: 801, dtype: int64

# Year

In [46]:
len(data.year.unique())

250

In [47]:
data.year.value_counts()

2015.0    143
2017.0    136
2016.0    130
2011.0    128
2014.0    127
         ... 
1786.0      1
1785.0      1
1784.0      1
1783.0      1
1815.0      1
Name: year, Length: 249, dtype: int64

In [48]:
data.year.isnull().value_counts()

False    6300
True       12
Name: year, dtype: int64

# Fatal

In [49]:
data['fatal_(y/n)'].value_counts()

N          4293
Y          1388
UNKNOWN      71
 N            7
M             1
2017          1
N             1
y             1
Name: fatal_(y/n), dtype: int64

In [50]:
data['fatal_(y/n)'] = data['fatal_(y/n)'].str.strip()

In [51]:
data['fatal_(y/n)'].value_counts()

N          4301
Y          1388
UNKNOWN      71
M             1
2017          1
y             1
Name: fatal_(y/n), dtype: int64

In [52]:
data['fatal_(y/n)'].replace(['M','2017','UNKNOWN','y'], np.nan, inplace=True)

In [53]:
data['fatal_(y/n)'].replace(['N'], 0, inplace=True)
data['fatal_(y/n)'].replace(['Y'], 1, inplace=True)

In [54]:
data['fatal_(y/n)'].value_counts()

0.0    4301
1.0    1388
Name: fatal_(y/n), dtype: int64

# Sex

In [55]:
data.sex.value_counts()

M      5094
F       637
M         2
N         2
lli       1
.         1
Name: sex, dtype: int64

In [56]:
data.sex = data.sex.str.strip()
data.sex.replace(['N','.','lli'],np.nan,inplace=True)
data.sex.value_counts()

M    5096
F     637
Name: sex, dtype: int64

# Age

In [57]:
data.age.value_counts()

17                154
18                150
19                142
20                141
15                139
                 ... 
20?                 1
 28                 1
7      &    31      1
 30                 1
13 or 14            1
Name: age, Length: 157, dtype: int64

In [58]:
data.age.isnull().value_counts()

False    3471
True     2841
Name: age, dtype: int64

In [59]:
data.age = pd.to_numeric(data.age, errors='coerce')

In [60]:
data.age.isnull().value_counts()

False    3369
True     2943
Name: age, dtype: int64

# Activity

In [61]:
data.activity = data.activity.str.upper()
data.activity = data.activity.str.strip()

In [62]:
data.activity.value_counts().head(30)

SURFING               976
SWIMMING              916
FISHING               444
SPEARFISHING          340
BATHING               165
WADING                149
DIVING                130
STANDING              101
SCUBA DIVING           91
SNORKELING             89
BODY BOARDING          65
BODY SURFING           50
BOOGIE BOARDING        45
KAYAKING               35
FELL OVERBOARD         33
TREADING WATER         32
PEARL DIVING           32
FREE DIVING            30
SURF SKIING            19
WINDSURFING            19
WALKING                17
SHARK FISHING          16
FLOATING               15
ROWING                 14
CANOEING               14
KAYAK FISHING          14
SEA DISASTER           14
SURF-SKIING            12
SURF FISHING           12
FISHING FOR SHARKS     11
Name: activity, dtype: int64

In [63]:
data.activity = data.activity.apply(lambda x: 'FISHING' if 'FISH' in str(x) else x)
data.activity = data.activity.apply(lambda x: 'SURFING' if 'SURF' in str(x) else x)
data.activity = data.activity.apply(lambda x: 'SURFING' if 'BODY' in str(x) else x)
data.activity = data.activity.apply(lambda x: 'KAYAKING' if 'CANOE' in str(x) else x)
data.activity = data.activity.apply(lambda x: 'SWIMMING' if 'SWIM' in str(x) else x)
data.activity = data.activity.apply(lambda x: 'SCUBA DIVING' if 'DIVING' in str(x) else x)
data.activity = data.activity.apply(lambda x: 'BOATING' if 'BOAT' in str(x) else x)
data.activity = data.activity.apply(lambda x: 'BATHING' if 'TREADING WATER'  in str(x) else x)
data.activity = data.activity.apply(lambda x: 'BOATING' if 'SAIL' in str(x) else x)
data.activity = data.activity.apply(lambda x: 'PADDLE BOARDING' if 'PADDL'  in str(x) else x)
data.activity = data.activity.apply(lambda x: 'SEA DISASTER' if 'DISASTER'  in str(x) else x)

data.activity = data.activity.replace('BOOGIE BOARDING', 'SURFING')
data.activity = data.activity.replace('ROWING', 'KAYAKING')
data.activity = data.activity.replace('FELL OVERBOARD', 'FELL INTO THE WATER')

In [64]:
data.activity.value_counts().head()

SURFING         1347
FISHING         1225
SWIMMING        1091
SCUBA DIVING     497
BATHING          202
Name: activity, dtype: int64

# Species

In [65]:
data.species = data.species.str.strip()
data.species = data.species.str.upper()

In [71]:
lista_de_especies = ['WHITE','TIGER','BULL','BLACKTIP','NURSE','BLUE','REEF','AKO','RAGGEDTOOTH','HAMMERHEAD','BRONZE','LEMON','ZAMBESI','MAKO','WHALE','WOBBEGONG','SAND','SPINNER','SEVENGILL','BASKING','DUSKY','ANGEL','CARPET','GALAPAGOS','SILKY','PORBEAGLE','GILL','DOG','GREY','SALMON','SANDBAR','GILL','COPPER','SHOVELNOSE','THRESHER', 'SILVERTIP']

In [72]:
def limpiar_species(s, lst):
    # Verifica si alguna palabra en lst está en s
    for word in lst:
        if word in s:
            return word + ' SHARK'
    return 'UNIDENTIFIED'    

In [73]:
data.species = data.species.apply(lambda x: limpiar_species(str(x), lista_de_especies))

In [86]:
data.species.value_counts().head()

UNIDENTIFIED      4405
WHITE SHARK        664
TIGER SHARK        283
BULL SHARK         177
BLACKTIP SHARK     101
Name: species, dtype: int64

# Original order

In [92]:
data.original_order = 0

# Time

In [96]:
data.head(20)

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018-06-25,2018-06-25,2018.0,BOAT,USA,CALIFORNIA,"Oceanside, San Diego County",PADDLE BOARDING,Julie Wolfe,F,57.0,"No injury to occupant, outrigger canoe and pad...",0.0,18h00,WHITE SHARK,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,0,NaN,NaN
1,2018-06-18,2018-06-18,2018.0,UNPROVOKED,USA,GEORGIA,"St. Simon Island, Glynn County",STANDING,Adyson McNeely,F,11.0,Minor injury to left thigh,0.0,14h00 -15h00,UNIDENTIFIED,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-18,2018-06-18,0,NaN,NaN
2,2018-06-09,2018-06-09,2018.0,OTHER,USA,HAWAII,"Habush, Oahu",SURFING,John Denges,M,48.0,Injury to left lower leg from surfboard skeg,0.0,07h45,UNIDENTIFIED,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-09,2018-06-09,0,NaN,NaN
3,2018-06-08,2018-06-08,2018.0,UNPROVOKED,AUSTRALIA,NEW SOUTH WALES,Arrawarra Headland,SURFING,male,M,NaN,Minor injury to lower leg,0.0,NaN,UNIDENTIFIED,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,0,NaN,NaN
4,2018-06-04,2018-06-04,2018.0,PROVOKED,MEXICO,COLIMA,La Ticla,SCUBA DIVING,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,0.0,NaN,TIGER SHARK,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,0,NaN,NaN
5,2018-06-03,2018-06-03,2018.0,UNPROVOKED,AUSTRALIA,NEW SOUTH WALES,"Flat Rock, Ballina",SURFING,Chris,M,NaN,"No injury, board bitten",0.0,NaN,UNIDENTIFIED,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,0,NaN,NaN
6,2018-06-03,2018-06-03,2018.0,UNPROVOKED,BRAZIL,PERNAMBUCO,"Piedade Beach, Recife",SWIMMING,Jose Ernesto da Silva,M,18.0,FATAL,1.0,Late afternoon,TIGER SHARK,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,0,NaN,NaN
7,2018-05-27,2018-05-27,2018.0,UNPROVOKED,USA,FLORIDA,"Lighhouse Point Park, Ponce Inlet, Volusia County",FISHING,male,M,52.0,Minor injury to foot. PROVOKED INCIDENT,0.0,NaN,LEMON SHARK,"K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-27,2018-05-27,0,NaN,NaN
8,2018-05-26,2018-05-26,2018.0,UNPROVOKED,USA,FLORIDA,"Cocoa Beach, Brevard County",WALKING,Cody High,M,15.0,Lower left leg bitten,0.0,17h00,BULL SHARK,"K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,0,NaN,NaN
9,2018-05-26,2018-05-26,2018.0,UNPROVOKED,USA,FLORIDA,"Daytona Beach, Volusia County",STANDING,male,M,12.0,Minor injury to foot,0.0,14h00,UNIDENTIFIED,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,0,NaN,NaN


In [ ]:
#datos=datos.fillna('UNKNOWN') eliminar valores nulos